# Heat diffusion with Crank-Nicolson

In [1]:
import numpy as np, os, sys,subprocess, matplotlib
from pathlib import Path
from IPython.display import HTML
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm


matplotlib.rc('font', size=18)
matplotlib.rc('font', family='Arial')

In [2]:
#definition of numerical parameters
N = 51 #number of grid points
dt = 1.e-4 #time step
L = float(1) #size of grid
nsteps = 2000 #number of time steps
dx = L/(N-1) #grid spacing
nplot = 20 #number of timesteps before plotting

r = dt/dx**2 #assuming heat diffusion coefficient == 1

#initialize matrices A, B and b array
A = np.zeros((N-2,N-2))
B = np.zeros((N-2,N-2))
b = np.zeros((N-2))
#define matrices A, B and b array
for i in range(N-2):
    if i==0:
        A[i,:] = [2+2*r if j==0 else (-r) if j==1 else 0 for j in range(N-2)]
        B[i,:] = [2-2*r if j==0 else r if j==1 else 0 for j in range(N-2)]
        b[i] = 0. #boundary condition at i=1
    elif i==N-3:
        A[i,:] = [-r if j==N-4 else 2+2*r if j==N-3 else 0 for j in range(N-2)]
        B[i,:] = [r if j==N-4 else 2-2*r if j==N-3 else 0 for j in range(N-2)]
        b[i] = 0. #boundary condition at i=N
    else:
        A[i,:] = [-r if j==i-1 or j==i+1 else 2+2*r if j==i else 0 for j in range(N-2)]
        B[i,:] = [r if j==i-1 or j==i+1 else 2-2*r if j==i else 0 for j in range(N-2)]

#initialize grid
x = np.linspace(0,1,N)
#initial condition
u = np.asarray([2*xx if xx<=0.5 else 2*(1-xx) for xx in x])
#evaluate right hand side at t=0
bb = B.dot(u[1:-1]) + b

fig = plt.figure()
plt.plot(x,u,linewidth=2)

# Create frames directory
frames_dir = Path("frames")
frames_dir.mkdir(exist_ok=True)
filename = frames_dir / "foo0000.png"


fig.set_tight_layout(True);
plt.xlabel("x")
plt.ylabel("u")
plt.title("t = 0")
plt.savefig(filename,dpi=200,bbox_inches='tight')
plt.clf()

c = 0
for j in tqdm(range(nsteps), desc="Time stepping", unit="step"):
    #print(j)
    #find solution inside domain
    u[1:-1] = np.linalg.solve(A,bb)
    #update right hand side
    bb = B.dot(u[1:-1]) + b
    if(j%nplot==0): #plot results every nplot timesteps
        plt.plot(x,u,linewidth=2)
        plt.ylim([0,1])
        filename = frames_dir / f"foo{str(c+1).zfill(4)}.png"
        plt.xlabel("x")
        plt.ylabel("u")
        plt.title("t = %2.2f"%(dt*(j+1)))
        plt.savefig(filename,dpi=200,bbox_inches='tight')
        
        plt.clf()
        c += 1


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

<Figure size 640x480 with 0 Axes>

In [ ]:
cmd = [
    "ffmpeg", "-y",
    "-i", str(frames_dir / "foo%04d.png"),
    "out.mp4"
]
subprocess.check_call(cmd)
os.system("rm -f *.png")

In [ ]:
for png in frames_dir.glob("*.png"):
    png.unlink()

In [ ]:
HTML("""
    <video alt="test" width="620" controls>
        <source src="out.mp4" type="video/mp4">
    </video>
""")